# F1 Analysis - Introduction to Data Sciences project

## Importing libraries

In [1]:
# Already added some that we most probably will use
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# Add more whenever needed
# ...
from datetime import datetime

## Reading in the data

The data consists of most of the freely available Formula 1 data from 1950 to 2018. <br>
Link: http://ergast.com/mrd/

In [2]:
circuits = pd.read_csv('./f1db_csv/circuits.csv', header=None)
constructor_results = pd.read_csv('./f1db_csv/constructor_results.csv', header=None)
constructor_standings = pd.read_csv('./f1db_csv/constructor_standings.csv', header=None)
constructors = pd.read_csv('./f1db_csv/constructors.csv', header=None)
drivers = pd.read_csv('./f1db_csv/driver.csv', header=None)
driver_standings = pd.read_csv('./f1db_csv/driver_standings.csv', header=None)
lap_times = pd.read_csv('./f1db_csv/lap_times.csv', header=None)
pit_stops = pd.read_csv('./f1db_csv/pit_stops.csv', header=None)
qualifying = pd.read_csv('./f1db_csv/qualifying.csv', header=None)
races = pd.read_csv('./f1db_csv/races.csv', header=None)
results = pd.read_csv('./f1db_csv/results.csv', header=None)
seasons = pd.read_csv('./f1db_csv/seasons.csv', header=None)
status = pd.read_csv('./f1db_csv/status.csv', header=None)

## Adding correct column names to the datasets

Column names are based on the column names from here: https://www.kaggle.com/cjgdev/formula-1-race-data-19502017. <br>
Column names in our project are in snake case (snake_case).

In [3]:
circuits_column_names = ['circuit_id', 'circuit_ref', 'name', 'location', 'country', 'lat', 'lng', 'alt', 'url']
constructor_results_column_names = ['constructor_results_id', 'race_id', 'constructor_id', 'points', 'status']
constructor_standings_column_names = ['constructor_standings_id', 'race_id', 'constructor_id', 'points', 'position', 'position_text', 'wins']
constructors_column_names = ['constructor_id', 'constructor_ref', 'name', 'nationality', 'url']
drivers_column_names = ['driver_id', 'driver_ref', 'number', 'code', 'forename', 'surname', 'birth_date', 'nationality', 'url']
driver_standings_column_names = ['driver_standings_id', 'race_id', 'driver_id', 'points', 'position', 'position_text', 'wins']
lap_times_column_names = ['race_id', 'driver_id', 'lap', 'position', 'time', 'milliseconds']
pit_stops_column_names = ['race_id', 'driver_id', 'stop', 'lap', 'time', 'duration', 'milliseconds']
qualifying_column_names = ['qualify_id', 'race_id', 'driver_id', 'constructor_id', 'number', 'position', 'q1', 'q2', 'q3']
races_column_names = ['race_id', 'year', 'round', 'circuit_id', 'name', 'date', 'time', 'url']
results_column_names = ['result_id', 'race_id', 'driver_id', 'constructor_id', 'number', 'grid', 'position', 'position_text', 'position_order', 'points', 'laps', 'time', 'milliseconds', 'fastest_lap', 'rank', 'fastest_lap_time', 'fastest_lap_speed', 'status_id']
seasons_column_names = ['year', 'url']
status_column_names = ['status_id', 'status']

In [4]:
circuits.columns = circuits_column_names
constructor_results.columns = constructor_results_column_names
constructor_standings.columns = constructor_standings_column_names
constructors.columns = constructors_column_names
drivers.columns = drivers_column_names
driver_standings.columns = driver_standings_column_names
lap_times.columns = lap_times_column_names
pit_stops.columns = pit_stops_column_names
qualifying.columns = qualifying_column_names
races.columns = races_column_names
results.columns = results_column_names
seasons.columns = seasons_column_names
status.columns = status_column_names

## Replacing different types of non-existent (and non-sensical) values with a universal one

We are using np.nan as the universal non-existent value.

In [5]:
datasets = [circuits, constructor_results, constructor_standings, constructors, drivers, driver_standings, lap_times, pit_stops, qualifying, races, results, seasons, status]

for data in datasets:
    for column in data.columns:
        data[column] = data[column].replace('\\N', np.nan)  # Replacing previously used non-existent value with ours
        data[column] = data[column].replace('', np.nan)  # Replacing missing values

## Comparing the fastest lap times at frequent circuits during the 2004-2018 period

We are using the absolute fastest lap times at every track year by year (fastest lap during the race). <br>
There are 14 such circuits. <br>
Afterwards we shall make a 2x7 grid of the corresponding graphs.

In [6]:
years = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

suitable_races = []

for index, row in races.iterrows():
    year = int(row['year'])
    if (year in years):  # If the year of the race is between 2004 and 2018
        suitable_races.append([row['race_id'], row['circuit_id'], year])

circuit_year_lap = []

for race in suitable_races:
    race_id = race[0]
    race_lap_times = results.loc[results['race_id'] == race_id]['fastest_lap_time']
    lowest_race_lap_time = 10000
    for lap_time in race_lap_times:
        if (not lap_time is np.nan):
            minutes_seconds = lap_time.split(":")
            lap_time = float(int(minutes_seconds[0]) * 60 + float(minutes_seconds[1]))
            if (lap_time < lowest_race_lap_time):
                lowest_race_lap_time = lap_time
    circuit_year_lap.append([race[1], race[2], lowest_race_lap_time])

for i in circuit_year_lap:
    circuit_id = i[0]
    circuit_name = circuits.loc[circuits['circuit_id'] == circuit_id]['name']
    i[0] = circuit_name.tolist()[0]
    
df_circuit_year_lap = pd.DataFrame(circuit_year_lap)
df_circuit_year_lap.columns = ['circuit', 'year', 'fastest lap time (s)']

df_circuit_year_lap = df_circuit_year_lap.sort_values(['circuit', 'year'], ascending=True)
    

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
df_circuit_year_lap = df_circuit_year_lap.groupby('circuit').filter(lambda x : len(x)>10)
print(df_circuit_year_lap)

                            circuit  year  fastest lap time (s)
89   Albert Park Grand Prix Circuit  2004  84.125              
70   Albert Park Grand Prix Circuit  2005  85.683              
54   Albert Park Grand Prix Circuit  2006  86.045              
35   Albert Park Grand Prix Circuit  2007  85.235              
17   Albert Park Grand Prix Circuit  2008  87.418              
0    Albert Park Grand Prix Circuit  2009  87.706              
108  Albert Park Grand Prix Circuit  2010  88.358              
126  Albert Park Grand Prix Circuit  2011  88.947              
145  Albert Park Grand Prix Circuit  2012  89.187              
165  Albert Park Grand Prix Circuit  2013  89.274              
184  Albert Park Grand Prix Circuit  2014  92.478              
208  Albert Park Grand Prix Circuit  2015  90.945              
222  Albert Park Grand Prix Circuit  2016  88.997              
243  Albert Park Grand Prix Circuit  2017  86.538              
263  Albert Park Grand Prix Circuit  201

## Predicting the race winners of the 2018 season based on pre-race and race data (except for finishing positions) from 2014-2017

2018 is the last season of which we have data. <br>
2014-2017 (and 2018) are the years following regulation (rules) changes after 2013 so the data should be somewhat similiar.

Because the data is from the past we know that only racers from the top 3 teams (Mercedes, Ferrari, Red Bull) have won races during this period, thus we will focus only on the 6 drivers in these teams (each team has 2 drivers in every race).

It's worth noting that all 6 drivers were in those teams for only the 2017 season. Earlier on some of them were in different teams. This means we have to use data from their earlier races for other teams as well.

The top 6 drivers during 2018: <br>
Red Bull
* Max Verstappen
* Daniel Ricciardo

Ferrari
* Sebastian Vettel
* Kimi Räikkönen

Mercedes
* Lewis Hamilton
* Valtteri Bottas

Info to use
* Circuit (id)
* Season
* Constructor (id)
* Pit stops
* Pit stop duration (duration in dataset * amount of pit stops)
* Qualifying position (from qualifying dataset)
* Q3 time (if it doesn't exist use the slowest of the rest of the top 6, because they probably had a mechanical issue and thus the time would ruin predictions)
* Grid position (from results dataset) (doesn't have to match qualifying position, i.e. they got penalties for swapping parts after the qualifying)
* On which lap they got the fastest lap (from results dataset)
* Time of the fastest lap (from results dataset)
* Round (which race of the year was it, i.e. the 5th race out of 22) (from races dataset)

We will create a training dataset from years 2014-2017. <br>
The test dataset will be of the year 2018.

In [7]:
years = [2014, 2015, 2016, 2017, 2018]
# Max Verstappen, Daniel Ricciardo, Sebastian Vettel, Kimi Räikkönen, Lewis Hamilton, Valtteri Bottas
driver_ids = [830, 817, 20, 8, 1, 822]

suitable_races = []
for index, row in races.iterrows():
    year = int(row['year'])
    if (year in years):  # If the year of the race is between 2014 and 2018
        suitable_races.append([row['race_id'], year, row['circuit_id'], row['round']])
        

# Suitable personal results
suitable_results = []
for race in suitable_races:
    race_id = race[0]
    suitable_race_all_results = results.loc[results['race_id'] == race_id]
    for driver_id in driver_ids:
        suitable_race_driver_results = suitable_race_all_results.loc[suitable_race_all_results['driver_id'] == driver_id]
        # The driver did indeed take part in this race
        if (suitable_race_driver_results.empty == False):
            # Converting Series to a list and then taking the first (and only) element from the list (the 'result_id')
            result_id = suitable_race_driver_results['result_id'].tolist()[0]
            constructor_id = suitable_race_driver_results['constructor_id'].tolist()[0]
            grid_position = suitable_race_driver_results['grid'].tolist()[0]
            lap_nr_of_fastest_lap = suitable_race_driver_results['fastest_lap'].tolist()[0]
            fastest_lap_time = suitable_race_driver_results['fastest_lap_time'].tolist()[0]
            result_data_to_append = [result_id, race_id, driver_id, constructor_id, grid_position, 
                              lap_nr_of_fastest_lap, fastest_lap_time]
            suitable_results.append(result_data_to_append)          
#print(suitable_results)


# At this point we have the result_id, race_id, driver_id, constructor_id, grid_position, 
# lap_nr_of_fastest_lap, fastest_lap_time

# We still need values for: 
# circuit, 
# year, 
# amount of pit stops, 
# pit stop duration, 
# qualifying position,
# Q3 time, 
# round (which race of the year was it)

df_results = pd.DataFrame(suitable_results)
df_results.columns = ['result_id', 'race_id', 'driver_id', 'constructor_id', 'grid_position', 
                      'lap_of_fastest_lap', 'fastest_lap_time']

# Adding the missing columns (with np.nan)
# Using np.nan some int values turn into floats
# This is one fix for some values (but not all):
# (with -1 as the value so if we later add int-s they won't be made into floats)
# This can also be fixed later and for the machine learning this should not matter at all
unadded_columns = ['circuit', 'year', 'pit_stops', 'pit_stop_duration', 'qualifying_position', 'Q3_time', 'round']
for column in unadded_columns:
    df_results[column] = np.nan

#print(df_results)

# Adding values for columns: year, circuit, round, qualifying position, Q3 time, amount of pit stops, pit stop duration
for index, row in df_results.iterrows():
    race_id = int(row['race_id'])
    
    # Adding values for columns: 'circuit', 'year', 'round'
    for race in suitable_races:
        if (race[0] == race_id):
            year = int(race[1])
            circuit_id = int(race[2])
            round = int(race[3])
            df_results.loc[index, ['circuit']] = circuit_id
            df_results.loc[index, ['year']] = year
            df_results.loc[index, ['round']] = round
    
    driver_id = int(row['driver_id'])
    
    # Adding values for columns: 'qualifying_position', 'Q3_time'
    qualifying_data = qualifying.loc[(qualifying['race_id'] == race_id) & (qualifying['driver_id'] == driver_id)]
    # Converting Series to a list (which only contains one element or none)
    qualifying_position = qualifying_data['position'].tolist()
    if not qualifying_position:  # The qualifying position field is empty
        df_results.loc[index, ['qualifying_position']] = np.nan
    else:
        df_results.loc[index, ['qualifying_position']] = qualifying_position[0]
    q3_time = qualifying_data['q3'].tolist()
    if not q3_time:  # The Q3 time field is empty
        df_results.loc[index, ['Q3_time']] = np.nan
    else:
        df_results.loc[index, ['Q3_time']] = qualifying_data['q3'].tolist()[0]
        
    # Adding values for columns: 'pit_stops', 'pit_stop_duration'
    pit_stop_data = pit_stops.loc[(pit_stops['race_id'] == race_id) & (pit_stops['driver_id'] == driver_id)]
    # Converting Series to a list (which only contains one element or none)
    pit_stops_amount = pit_stop_data['stop'].tolist()
    #
    # NEEDS TO BE CHANGED
    # pit_stop_data contains info of which pitstop of a driver was it (for example the 2nd pitstop during the race)
    # and how long it took.
    # Therefore we need to loop over it and add together all the durations and mark how many pit stops a driver did
    #
    if not pit_stops_amount:  # The pit stops amount field is empty
        df_results.loc[index, ['pit_stops']] = np.nan
        df_results.loc[index, ['pit_stop_duration']] = np.nan
    else:
        df_results.loc[index, ['pit_stops']] = pit_stops_amount[0]
        pit_stop_duration = pit_stop_data['milliseconds'].tolist()
        if not pit_stop_duration:  # The pit stop duration field is empty
            df_results.loc[index, ['pit_stop_duration']] = np.nan
        else:
            df_results.loc[index, ['pit_stop_duration']] = float(pit_stop_duration[0]) * int(pit_stops_amount[0])

#print(df_results)   

# Turning Q3_time and fastest_lap_time into seconds, pit_stop_duration from milliseconds to seconds
for index, row in df_results.iterrows():
    # Replacing Q3_time from string to seconds
    Q3_time = row['Q3_time']
    if (not Q3_time is np.nan):
        minutes_seconds = Q3_time.split(":")
        Q3_time = float(int(minutes_seconds[0]) * 60 + float(minutes_seconds[1]))
        df_results.loc[index, ['Q3_time']] = Q3_time
    
    # Replacing fastest_lap_time from string to seconds
    fastest_lap_time = row['fastest_lap_time']
    if (not fastest_lap_time is np.nan):
        minutes_seconds = fastest_lap_time.split(":")
        fastest_lap_time = float(int(minutes_seconds[0]) * 60 + float(minutes_seconds[1]))
        df_results.loc[index, ['fastest_lap_time']] = fastest_lap_time
        
    # Converting pit stop duration from milliseconds to seconds
    pit_stop_duration = row['pit_stop_duration']
    if (not pit_stop_duration is np.nan):
        df_results.loc[index, ['pit_stop_duration']] = pit_stop_duration / 1000.0

#print(df_results)

# Converting unnecessary floats to ints
float_to_int_columns = ['circuit', 'year', 'round']
df_results[float_to_int_columns] = df_results[float_to_int_columns].applymap(np.int64)
# Can't convert qualifying_position and pit_stops from float to int, because they also include np.nan and int columns don't
# support np.nan

print(df_results)

# Replace np.nan values with sensible values
# For example, a missing Q3 time should perhaps be replaced with the slowest of the rest of the top 6.
# Because when a top 6 driver doesn't get into Q3, they probably had an issue with the car, so it ruins the prediction
# due to factors that were not under the drivers control (too random)

# Then we should one-hot encode categorical fields (even though they are represented by integers, like 'constructor_id')

     result_id  race_id  driver_id  constructor_id  grid_position  \
0    22151      900      817        9               2               
1    22147      900      20         9               12              
2    22136      900      8          6               11              
3    22148      900      1          131             1               
4    22134      900      822        3               15              
5    22167      901      817        9               5               
6    22154      901      20         9               2               
7    22163      901      8          6               6               
8    22152      901      1          131             1               
9    22159      901      822        3               18              
10   22177      902      817        9               13              
11   22179      902      20         9               10              
12   22183      902      8          6               5               
13   22174      902      1        